# Blending optimization tutorial with linear programming in Python
Based on: https://github.com/13ff6/CementOptFF/blob/main/CementBlendOpt_FF.py

Also see article: https://towardsdatascience.com/blending-optimization-tutorial-with-linear-programming-in-python-74bcc443e4e5

Refactored from original:
- Add variables x[4]..x[9] to represent q1..q6 discussed in the article. Note that their objective function coefficients equal zero.
- Substitute x[4]..x[9] into the original constraints to make them much simpler and less repetitive.
- Include `1 *` in `eq` constraints, for consistency with `ineq` constraints.
- Add spaces to the data and constraints to emphasise the matrix nature of SciPy's model representation.
- Add formatted output of the results.

In [1]:
import pandas as pd
from scipy.optimize import minimize

In [2]:
#                   x[0]    x[1]    x[2]    x[3]    x[4]    x[5]    x[6]    x[7]    x[8]    x[9]
Cost = pd.Series([ 20.00,   6.00, 230.00, 160.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00]) # price of each material in order
X_0  = pd.Series([  0.25,   0.25,   0.25,   0.25,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00]) # initial guess 25% of each material

def cost_fun(x):
    return sum(Cost*x) # minimize sum of material costs

cons = (
        {'type': 'eq',   'fun': lambda x:  1 * (11.46*x[0] + 56.30*x[1] +  1.64*x[2] +  5.36*x[3] -  1.00*x[4]                                                                  +  0.00)},
        {'type': 'eq',   'fun': lambda x:  1 * ( 1.62*x[0] + 10.18*x[1] + 39.06*x[2] +  2.99*x[3]              -  1.00*x[5]                                                     +  0.00)},
        {'type': 'eq',   'fun': lambda x:  1 * ( 0.96*x[0] +  4.76*x[1] + 33.94*x[2] + 83.61*x[3]                           -  1.00*x[6]                                        +  0.00)},
        {'type': 'eq',   'fun': lambda x:  1 * (48.36*x[0] + 12.70*x[1] + 12.20*x[2] +  1.84*x[3]                                        -  1.00*x[7]                           +  0.00)},
        {'type': 'eq',   'fun': lambda x:  1 * ( 0.02*x[0] +  0.22*x[1] +  0.00*x[2] +  2.42*x[3]                                                     -  1.00*x[8]              +  0.00)},
        {'type': 'eq',   'fun': lambda x:  1 * (31.00*x[0] + 12.12*x[1] +  6.77*x[2] +  2.08*x[3]                                                                  -  1.00*x[9] +  0.00)},
    
        {'type': 'eq',   'fun': lambda x:  1 * ( 1.00*x[0] +  1.00*x[1] +  1.00*x[2] +  1.00*x[3]                                                                               -  1.00)},
    
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                     1.00*x[4] -  3.00*x[5] -  3.00*x[6]                                        +  0.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                    -1.00*x[4] +  2.80*x[5] +  2.80*x[6]                                        +  0.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                                  1.00*x[5] -  2.50*x[6]                                        +  0.00)},
        {'type': 'ineq', 'fun': lambda x:  1 * (                                                                  1.00*x[5] -  1.50*x[6]                                        +  0.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                    -2.80*x[4] -  1.10*x[5] -  0.70*x[6] +  1.05*x[7]                           +  0.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                     2.80*x[4] +  1.10*x[5] +  0.70*x[6] -  1.11*x[7]                           +  0.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                    -7.60*x[4] -  6.72*x[5] -  1.43*x[6] +  4.07*x[7] -  2.85*x[8] +  0.70*x[9] - 70.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                     7.60*x[4] +  6.72*x[5] +  1.43*x[6] -  4.07*x[7] +  2.85*x[8] -  0.50*x[9] + 50.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                     8.60*x[4] +  5.07*x[5] +  1.08*x[6] -  3.04*x[7] +  2.15*x[8] +  0.40*x[9] - 40.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                    -8.60*x[4] -  5.07*x[5] -  1.08*x[6] +  3.04*x[7] -  2.15*x[8] -  2.00*x[9] + 20.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                                  2.65*x[5] -  1.69*x[6]                           -  0.10*x[9] - 10.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                                               3.04*x[6]                           +  0.12*x[9] - 12.00)},
        {'type': 'ineq', 'fun': lambda x: -1 * (                                                                            -  3.04*x[6]                           -  0.07*x[9] -  7.00)}
       )

res = minimize(cost_fun, X_0, method='SLSQP',constraints=cons) # optimize

In [3]:
print('Status:   ', res.message)
if res.success:
    print(f'Objective: ${res.fun:.2f}')
    Ingredients = pd.DataFrame()
    for i in range(0, 4):
        Ingredients.loc[i, 'Mix'] = res.x[int(i)]
    Ingredients.index = ['Linestone', 'Clay', 'Bauxite', 'Pyrites']
    Ingredients.loc["Total"] = Ingredients.sum()
    display(Ingredients.style.format({'Mix': "{:.2%}"}))    
else:
    print('No solution')

Status:    Optimization terminated successfully
Objective: $22.66


,Mix
Linestone,88.09%
Clay,9.91%
Bauxite,1.79%
Pyrites,0.20%
Total,100.00%
